In [94]:
%%pycodestyle
import numpy as np
import pandas as pd
# import math

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
# from sklearn.linear_model import LinearRegression
# from sklearn.metrics import r2_score, mean_squared_error
# from sklearn.feature_selection import SelectKBest
# from sklearn.feature_selection import f_regression

# import collections

%matplotlib inline

19:1: W391 blank line at end of file


# Overview

In [14]:
%%pycodestyle
pd.set_option('display.max_rows', 150)

3:1: W391 blank line at end of file


In [15]:
%%pycodestyle
df = pd.read_csv('listings.csv')
df.head(5)

4:1: W391 blank line at end of file


In [16]:
%%pycodestyle
df.price.describe()

3:1: W391 blank line at end of file


In [73]:
%%pycodestyle
df.describe()

3:1: W391 blank line at end of file


In [76]:
%%pycodestyle
# The shape meaning rows and columns of the dataframe
print(df.shape, '\n' + '#' * 60)

# The data types in the dataset
print(df.dtypes.value_counts(), '\n' + '#' * 60)


# The column names for the data type float64
print(df.dtypes[df.dtypes == 'float64'], '\n' + '#' * 60)

# The column names for the data type int
print(df.dtypes[df.dtypes == 'int64'], '\n' + '#' * 60)

# The column names for the data type object
print(df.dtypes[df.dtypes == 'object'], '\n' + '#' * 60)

17:1: W391 blank line at end of file


There are some columns that are of type object, but can be converted to int64 or float 64 because in fact the are numbers like price columns. be treated as integer or float (especially those price columns).

We keep that in mind for later, if we use some of them we first convert them into the right type.

##### Missing Values

In [20]:
%%pycodestyle
df.isnull().sum().sort_values(ascending=False)

3:1: W391 blank line at end of file


##### Sole Unique No Missing Values

In [22]:
%%pycodestyle
df.nunique()[df.nunique() == df.shape[0]]

3:1: W391 blank line at end of file


## Analysis

### Focus on the variables with no missing data

In [31]:
%%pycodestyle
df_sub = df[['price', 'extra_people', 'guests_included', 'accommodates', 'availability_30', 'room_type', 'bed_type', 'beds', 'bathrooms', 'amenities']]

df_sub

2:80: E501 line too long (151 > 79 characters)
5:1: W391 blank line at end of file


In [82]:
%%pycodestyle
def price_to_float(input_df, column_list):
    """_summary_

    Args:
        input_df (df): input dataframe that should be manipulated
        column_list (list): A list of column names that values should be converted to the float

    Returns:
        dataframe: the input dataframe with the input columns converted to float
    """
    output_df = input_df
    for col in column_list:
        print(col)
        if input_df.dtypes[col] == object:
            output_df[col] = input_df[col].str.replace('$', '').str.replace('€', '').str.replace(',', '')
            output_df[col].fillna(0, inplace=True)
            output_df[col] = output_df[col].astype(float)
    return output_df

7:80: E501 line too long (95 > 79 characters)
10:80: E501 line too long (80 > 79 characters)
16:80: E501 line too long (105 > 79 characters)
20:1: W391 blank line at end of file


In [83]:
%%pycodestyle
df_clean = price_to_float(df_sub, ['price', 'extra_people'])
df_clean['price']

4:1: W391 blank line at end of file


#### Question 1: Are accommodations with more bedrooms in general cheaper per person?

In [84]:
%%pycodestyle
# the histogram of the data
n, bins, patches = plt.hist(df.accommodates, 50, density=False, facecolor='g', alpha=0.75)

plt.xlabel('Accommodates')
plt.ylabel('Count')
plt.title('Histogram of accommodates')
plt.grid(True)
plt.show()

3:80: E501 line too long (90 > 79 characters)
10:1: W391 blank line at end of file


In [93]:
%%pycodestyle
def calcfullybookedpice(input_df):
    """
    Args:
        input_df (df): the df that should be manipulated

    Returns:
        df: the manipulated input dataframe
    """
    resultList = list()
    for row in range(input_df.shape[0]):
        extra_charge = (input_df['accommodates'][row]-input_df['guests_included'][row]) * input_df['extra_people'][row]
        fully_booked_price = input_df['price'][row] + extra_charge
        resultList.append(fully_booked_price)
    input_df["fully_booked_price"] = resultList
    return input_df

12:80: E501 line too long (119 > 79 characters)
17:1: W391 blank line at end of file


In [85]:
%%pycodestyle
df_clean = calcfullybookedpice(df_clean)
df_clean['PricePerPerson'] = df_clean['fully_booked_price']/df_clean['accommodates']

3:80: E501 line too long (84 > 79 characters)
4:1: W391 blank line at end of file


In [86]:
%%pycodestyle
p = sns.catplot(data=df_clean, x="beds", y="PricePerPerson", kind="boxen")
p.set_ylabels("PricePerPerson (in $)")
p.set_titles("Bedprices per Person")

5:1: W391 blank line at end of file


Based on the simple analysis seen in the boxplot above, it does not appear that more bedrooms generally lower the price per person. However, it can be seen that the variance decreases with a higher bed size.

#### Question 2: Which amenities are most offered in apartments?

In [71]:
%%pycodestyle
def countListOfStrings(df, searchColumn):
    """
    Args:
        df (dataframe): _description_
        searchColumn (str): _description_

    Returns:
        _type_: _description_
    """
    new_list = list()
    for row in range(df.shape[0]):
        arr = df[searchColumn][row].split(',')
        for v in arr:
            v_clean = v.replace('{', '').replace('}', '').replace('"', '')
            new_list.append(v_clean)
    return new_list


def plotCount(df, search_column):
    """
    Args:
        df (dataframe): _description_
        search_column (str"): _description_
    """
    temp_list = countListOfStrings(df, search_column)
    p = plt.figure(figsize=(12, 8))
    sns.countplot(y=countListOfStrings(df, search_column),
                  order=[elem for count, elem in sorted(((temp_list.count(e), e) for e in set(temp_list)), reverse=True)],
                  ).set(title="Countplot of " + search_column)

16:80: E501 line too long (122 > 79 characters)
18:1: W391 blank line at end of file


In [47]:
%%pycodestyle
plotCount(df_clean, 'amenities');

2:33: E703 statement ends with a semicolon
3:1: W391 blank line at end of file


As it can be seen in the count-plot above the most common amenities in the dataset listed are:
+ Wireless Internet
+ Heating
+ Kitchen
+ Internet
+ Smoke Detector

And the least common amenities listed are:

+ Lock on Bedroom Door
+ Doorman
+ Smoking Allowed
+ Other pet(s)
+ Washer / Dryer

Also in the lower area a blank thing is reported, but this is due to data errors and is omitted in further analysis.

#### Question 3: Is it possible to predict the Airbnb price by using a simple sub-sample of features in a basic Random Forest Regression?

#### Make ML dataframe subset

In [45]:
%%pycodestyle
df_model = df_clean.dropna()

3:1: W391 blank line at end of file


Drop the missing values since there are only 10 in the dataset not imputation methods are applied.

In [44]:
%%pycodestyle
# Get the output values that should be predicted
y = df_model['price']

# Get the feature values that should be used to predict
X = df_model[['extra_people', 'guests_included', 'accommodates', 'availability_30', 'room_type', 'bed_type', 'beds', 'bathrooms', 'amenities']]
X

6:80: E501 line too long (143 > 79 characters)
8:1: W391 blank line at end of file


In [42]:
%%pycodestyle
# limit to categorical data using df.select_dtypes()
X_cat = X.select_dtypes(include=[object])
X_cat

5:1: W391 blank line at end of file


#### Label encode

In [41]:
%%pycodestyle
# TODO: create a LabelEncoder object and fit it to each feature in X

# 1. INSTANTIATE
# encode labels with value between 0 and n_classes-1.
le = preprocessing.LabelEncoder()


# 2/3. FIT AND TRANSFORM
# use df.apply() to apply le.fit_transform to all columns
X_2 = X_cat.apply(le.fit_transform)
X_2.head()

13:1: W391 blank line at end of file


In [40]:
%%pycodestyle
X_non_cat = X.select_dtypes(exclude=[object])
X_non_cat

4:1: W391 blank line at end of file


#### Merge label encoded with other features

In [39]:
%%pycodestyle
X = pd.concat([X_non_cat, X_2], axis=1)
X.info()

4:1: W391 blank line at end of file


#### Split into train and test dataset

In [87]:
%%pycodestyle
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

2:80: E501 line too long (88 > 79 characters)
3:1: W391 blank line at end of file


#### Train the model

In [92]:
%%pycodestyle
from sklearn.ensemble import RandomForestRegressor

# create regressor object
regressor = RandomForestRegressor(n_estimators=100, random_state=20)

# fit the regressor with x and y data
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

10:1: W391 blank line at end of file


In [33]:
%%pycodestyle
sorted_idx = regressor.feature_importances_.argsort()
plt.barh(X_train.columns[sorted_idx], regressor.feature_importances_[sorted_idx])
plt.xlabel("Random Forest Feature Importance");

3:80: E501 line too long (81 > 79 characters)
4:47: E703 statement ends with a semicolon
5:1: W391 blank line at end of file


<b> ->  </b> Here the accommdates seems to be the most important variable, but this need to be considered carefully. Because the price is not the total price with all extra guest but the price for the included guests.

#### Calculate the loss after training 

In [32]:
%%pycodestyle
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

6:80: E501 line too long (86 > 79 characters)
7:1: W391 blank line at end of file


The price prediction model seems to be of poor quality and far from perfect. A standard error of 38 for a price of 127 on average is not very helpful. Moreover, the total standard deviation of the observed variables is 62. To further improve the model, the above considerations should be taken into account, as well as some more features should be engineered and perhaps some more or different features should be used. So, it can be said that by using a simple sub-set without deeper engineering using random forest regression, a good model fit cannot be obtained.

# References

- https://www.ritchieng.com/machinelearning-one-hot-encoding/
- https://medium.com/@theclickreader/random-forest-regression-explained-with-implementation-in-python-3dad88caf165
- https://mljar.com/blog/feature-importance-in-random-forest/